In [238]:
import json
import pandas as pd

read review_train, first process 10000

In [239]:
data_file = 'review_train.json'
sample = []
with open(data_file, 'r') as f:
    for line in f:
        sample.append(line.rstrip())   

In [240]:
for i in range(len(sample[:10000])):
        if i == 0:
            train10000 = pd.DataFrame(json.loads(sample[i]),index = [0])
        else:
            temp = pd.DataFrame(json.loads(sample[i]),index = [i])
            train10000 = train10000.append(temp)     

In [241]:
train10000.head()

,business_id,stars,text,date
0,31292,1.0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36
1,35344,5.0,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33
2,152538,5.0,I have to say that this office really has it t...,2016-11-09 20:09:03
3,71871,5.0,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38
4,64913,1.0,Today was my second out of three sessions I ha...,2018-01-30 23:07:38


first just retain English reviews

In [242]:
import langid
for i in range(10000):
        train10000.loc[i,'lang'] = langid.classify(train10000.text[i])[0]

In [243]:
train10000.lang.value_counts()

en    9954
fr      37
es       3
de       2
ja       1
zh       1
it       1
mg       1
Name: lang, dtype: int64

train_temp contains 9954 only English reviews

In [244]:
train_temp = train10000[train10000.lang == 'en']
train_temp.index = list(range(9954))

1.some common emoticons convert to words

In [245]:
import re
def convert_emo(text):
    re_happy = r"(:-\)|:\)|:-\]|:\]|:3|:c\)|:>|8\)|8-\)|:o\)|=\)|:\}|:\^\)|:-\)\))" 
    re_laugh = r"(:‑D|:D|8‑D|8D|x‑D|xD|X‑D|XD|=‑D|=D|=‑3|=3|B\^D)" 
    re_sad = r"(:‑\(|:\(|:‑c|:c|:‑<|:<|:‑\[|:\[|:\{|;\(|>:\[)" 
    re_angry = r"(:-\|\||:@|>:\()" 
    re_cry = r"(:'‑\(|:'\()" 
    re_horror = r"(D:<|D:|D8|D;|D=|DX|D‑':|v\.v)" 
    re_surprise = r"(>:O|:‑O|:O|:‑o|:o|8‑0|O_O|o‑o|O_o|o_O|o_o|O-O)" 
    re_kiss = r"(:\*|:-\*|:\^\*|\( '\}\{'\))" 
    re_wink = r"(;‑\)|;\)|\*-\)|\*\)|;‑\]|;\]|;D|;\^\)|:‑,)" 
    re_skeptical = r"(>:\\\\|>:/|:‑/|:‑\.|:/|:\\\\|=/|=\\\\|:L|=L|:S|>\.<)" 
    text = re.sub(re_happy,' happy ',text)
    text = re.sub(re_laugh,' laugh ',text)
    text = re.sub(re_sad,' sad ',text)
    text = re.sub(re_angry,' angry ',text)
    text = re.sub(re_cry,' cry ',text)
    text = re.sub(re_horror,' horror ',text)
    text = re.sub(re_surprise,' surprise ',text)
    text = re.sub(re_kiss,' kiss ',text)
    text = re.sub(re_wink,' wink ',text)
    text = re.sub(re_skeptical,' skeptical ',text)
    return text

In [246]:
convert_emo(">.< I am a pig. :D") #example

' skeptical  I am a pig.  laugh '

In [247]:
train_temp.text = train_temp.text.apply(convert_emo)

remove \\n (text = re.sub('\\n',' ',text)

In [251]:
def remove_gangn(text):
    text = re.sub('\n',' ',text)
    return text

In [252]:
train_temp.text = train_temp.text.apply(remove_gangn)

2.lowercase (str.lower())

3.first remove '_'  for need to add not_, and phrases later

In [254]:
def rm_underline(text):
    text = re.sub('_',' ',text.lower())
    return text

In [74]:
rm_underline('I am a _pig.') #example

'I am a  pig.'

In [255]:
train_temp.text = train_temp.text.apply(rm_underline)

4.convert n't to not, 'm to am, 's to is, 've to have, 'd to would

In [257]:
def convert_abb(text):
    text = re.sub('n\'t',' not',text)
    text = re.sub('\'m',' am',text)
    text = re.sub('\'s', ' is',text)
    text = re.sub('\'ve',' have',text)
    text = re.sub('\'d',' would',text)
    return text

In [106]:
convert_abb("I don't like pigs. I'm pig. I've pigs.") #example

'I do not like pigs. I am pig. I have pigs.'

In [258]:
train_temp.text = train_temp.text.apply(convert_abb)

5.add not_ 

In [260]:
import re
def add_negation(text):
    negation = ['not', 'never', 'lack', 'no','hardly','rarely']
    words = text.split()[:]
    flag = 0
    for index, word in enumerate(words):
        if(flag != 0):
            words[index] = 'not_' + words[index]
        if(re.match('.*[;,.?!\'\"]$', word)):
            flag = 0
        if(word in negation):
            flag = 1
    text = " ".join(words)
    return(text)

In [98]:
add_negation("I do not like pigs, but I like piggy.") #example

'I do not not_like not_pigs, but I like piggy.'

In [261]:
train_temp.text = train_temp.text.apply(add_negation)

6. remove punctuations : 

In [263]:
def remove_punc(text):
    text = re.sub('[/*-?!:;,\.\'\"\)\(]', ' ', text)
    return text

In [264]:
train_temp.text = train_temp.text.apply(remove_punc)

7. lemmatization

In [266]:
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
#nltk.download('averaged_perceptron_tagger')

In [274]:
def lemmatizer_all(text):
    new_text = []
    lemmatizer = WordNetLemmatizer()
    for word, tag in pos_tag(word_tokenize(text)):
        if tag.startswith('NN'):
            new_text.append(lemmatizer.lemmatize(word, pos='n'))
        elif tag.startswith('VB'):
            new_text.append(lemmatizer.lemmatize(word, pos='v'))
        elif tag.startswith('JJ'):
            new_text.append(lemmatizer.lemmatize(word, pos='a'))
        elif tag.startswith('R'):
            new_text.append(lemmatizer.lemmatize(word, pos='r'))
        else:
            new_text.append(word)
    return ' '.join(new_text)

In [268]:
' '.join(lemmatizer_all('I have many pigs. I cared their health, and they like swimming.')) #example

'I have many pig . I care their health , and they like swim .'

In [276]:
train_temp.text.loc[:100] = train_temp.text.loc[:100].apply(lemmatizer_all)

/Users/jiangyunwen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [305]:
train_temp.text.loc[:100]

0      total bill for this horrible service over $ gs...
1      i adore travis at the hard rock be new kelly c...
2      i have to say that this office really have it ...
3      go in for a lunch steak sandwich be delicious ...
4      today be my second out of three session i have...
5      i ll be the first to admit that i be not not_e...
6      tracy dessert have a big name in hong kong and...
7      this place have go down hill clearly they have...
8      it be a giant best buy with register i do not ...
9      like walk back in time every saturday morning ...
10     walk in around on a friday afternoon we sit at...
11     wow so surprised at the one and two star revie...
12     michael from red carpet vip be amaze i reach o...
13     i can not believe how thing have change in yea...
14     you ca not not_really not_find not_anything no...
15     great lunch today staff be very helpful in ass...
16     we have be a huge slim be fan since they open ...
17     good selection of class 

8.remove some not important words like 'to','in',.'he','she',......

In [329]:
no_importance_pos = ['IN','MD','PRP','PRP$','TO','WDT','WP','WP$','WRB']
no_importance_word = ['i','he','she','they','the','a','an','this','that','those','be']
only_text = []
for i in range(100):
    only_text.append(' '.join(train_temp.text.loc[i].split(' ')))

In [332]:
new_text = []
for sentence in only_text:
    retain = []
    POS = pos_tag(sentence.split(' '))
    for word in POS:
        if word[1] not in no_importance_pos and word[0] not in no_importance_word and len(word[0])!= 1:
            retain.append(word[0])
    new_text.append(' '.join(retain))    

9.phrases

In [334]:
from gensim.models.phrases import Phrases, Phraser

In [340]:
only_word = [sentence.split(' ') for sentence in new_text]

In [342]:
bigram = Phraser(Phrases(only_word, min_count=5, threshold=5))

In [343]:
final_text = []
for i in range(100):
    final_text.append(' '.join(bigram[only_word[i]]))

In [344]:
final_text

['total bill horrible service gs these crook actually have nerve charge pill check online pill have cent each avoid hospital er all cost',
 'adore travis hard rock new kelly cardenas salon always fan great blowout and no not_stranger not_to not_the not_chains not_offer not_this not_service however travis have take flawless blowout whole new level travis greets perfectly green swoosh otherwise perfectly style black hair and vega worthy rockstar outfit next come most relaxing and incredible shampoo get full head message cure even very bad migraine minute and scented shampoo room travis have freakishly strong finger good way and use perfect amount pressure superb then start glorious blowout not not_one not not_two but three people involve do best round brush action hair have ever see team stylist clearly get extremely well evident way talk and help one another really genuine and not not_some not_corporate not_requirement so much fun there next travis start flat iron way flip wrist get vol